In [1]:
# Do all imports for script. If this fails, you need to import more
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# from jupyter_datatables import init_datatables_mode
from csv import DictReader
import logging
from os import listdir
from datetime import datetime
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, DateTime, Date, MetaData, ForeignKey
from sqlalchemy.sql import text
from time import time
import hashlib

def find_database():
    names = []
    for file in listdir('Database'):
        if file.startswith('ETC_Droid_DB_'):
            names.append(f'Database/{file}')
    if len(names) == 0:
        time = datetime.now()
        return f'ETC_Droid_DB_{time.strftime("%Y%m%d%H%M%S")}.db'
    else:
        return names[-1]

print(f'Using database file: {find_database()}')
    
# Enable logging
handler = logging.FileHandler('sql.log')
handler.setLevel(logging.DEBUG)
logging.getLogger('sqlalchemy').addHandler(handler)

# Connect to the test.db sqlite database generated in the other file
engine = create_engine(f"sqlite:///{find_database()}", echo=False)
conn = engine.connect()

Using database file: Database/ETC_Droid_DB_20211108110401.db


In [2]:
def output_as_markdown_table(headers, values, center=True):
    # Get the amount of padding to add
    sizes = []
    for header in headers:
        sizes.append(len(header))
    for row in values:
        for index, value in enumerate(row):
            sizes[index] = max(sizes[index], len(str(value)))
    
    # Print the headers
    print('|', end='')
    for index, header in enumerate(headers):
        if center:
            print(f' {str(header).center(sizes[index])} |', end='')
        else:
            print(f' {str(header).ljust(sizes[index])} |', end='')
    print()
    
    # Print the separation row
    print('|', end='')
    for size in sizes:
        string = ''
        for x in range(size):
            string += '-'
        print(f' {string} |', end='')
    print()
    
    # Print the values
    for row in values:
        print('|', end='')
        for index, value in enumerate(row):
            if center:
                print(f' {str(value).center(sizes[index])} |', end='')
            else:
                print(f' {str(value).ljust(sizes[index])} |', end='')
        print()


def SequenceBuilder(listA):
    length = len(listA)
    sequence = []
    for i in range(length):
        sequence.append(i)
    return sequence


def queryTable(headers,values):
    sequence = SequenceBuilder(headers)
    #plt.rcParams['figure.figsize'] = [20, 2]
    #plt.rcParams['figure.dpi'] = 900
    fig, ax = plt.subplots(1,1)
    ax.axis('off')

    table = ax.table(cellText=values,
                     colLabels=headers,
                     cellLoc='left',
                     rowLoc='center',
                     loc="center")

    table.scale(1,2)
    table.auto_set_column_width(sequence)

    plt.show()

# Utility Functions

In [3]:
def format_number(number):
    number_string = str(number)
    
    output_string = ''
    while len(number_string) > 0:
        output_string = ',' + number_string[-3:] + output_string
        number_string = number_string[:-3]
    return output_string[1:]

print(format_number(10493160))
print(format_number(10493160) == '10,493,160')

10,493,160
True


## 'droid_ids' Table Contents
Droid profile output, ids are tied to unique files
### Header Information
1.  'id' -- A unqiue id number identifying this specific file
2.  'parent_id' -- The unqiue id number of the parent folder
3.  'uri' -- A uri that directs to the semester relative file location. Ex: file://2008_semester_1/wiixercise/file.zip
4.  'file_path' -- The semester relative path of this file. Ex.: /2008_semester_1/wiixercise/file.zip
5.  'filename' -- The name of this file inclusing the extension: file.zip
6.  'id_method' -- The method that was used to identify the file_format_name. Extension, Container or Signature
7.  'status' -- The status of the scanning of this file and its children. Done or Empty
8.  'size' -- The size of this file in bytes. If folder, then the size is blank
9.  'type' -- The type of item. File or Folder
10. 'file_extension' -- The file exenstion of the file. Blank for a folder
11. 'last_modified' -- The modification time of the file or folder
12. 'ext_mis_warning' -- True if a file extension does not match the binary signature of the file. False otherwise
13. 'hash' -- The sha256 hash of the file
14. 'file_format_count' -- The count of attached file format matches
15. 'project_name' -- The name of the project from ETC Past Projects Listing file. Will start with 'Unverified Name:' if a match could not be found.

## 'droid_formats' Table Contents
1. 'id' -- A unqiue id that identifies this specific format
2. 'file_id' -- The if of the file that this format belongs to.
3. 'pronom_id' -- PRONOM fmt number
4. 'mime_type' -- The detected MIME type. Blank if not found.
5. 'file_format_name' -- PRONOM Identified File Format 
6. 'file_format_version' -- PRONOM Identified File Version

### (Stats 1) Total Amounts Related to Files

Current DB Hash (11/08/2021): d4e91d4ec9b2187ad30fee77e9510842

1. Total Files: 10,493,160
2. Total Size: 16,082,869,474,251 bytes
3. Total File Extensions: 4,982
4. Total File Formats: 715

Current DB Hash (7/26/2021): c3c39eb0c842c852ee0517686d72772f

1. Total Files: 10,493,160
2. Total Size: 16,082,869,474,251 bytes
3. Total File Extensions: 4,982
4. Total File Formats: 10,580,350

Current DB Hash (7/22/2021): d3d9980c6153ef38f27ff46f9b2d356a

1. Total Files: 10,498,189
2. Total File Extensions: 4,982

Current DB Hash (7/20/2021): 5b838ed892dae08217083be9022ccd4e

1. Total Files: 10,498,189
2. Total File Extensions: 4,982

Current DB Hash (7/14/2021): 7e97ac65e748e1b0a5753bc95464a600 

1. Total Files: 10,444,003
2. Total File Extensions: 4,977

#### Current DB Hash Check

In [4]:
current_db_hash = "d4e91d4ec9b2187ad30fee77e9510842"

In [5]:
BLOCKSIZE = 65536
hasher = hashlib.md5()
with open(find_database(), 'rb') as afile:
    buf = afile.read(BLOCKSIZE)
    while len(buf) > 0:
        hasher.update(buf)
        buf = afile.read(BLOCKSIZE)

current_db_check_hash = hasher.hexdigest()
print(current_db_check_hash)

d4e91d4ec9b2187ad30fee77e9510842


In [6]:
if current_db_check_hash == current_db_hash:
    print("DB is current")
else:
    print("DB is not current")

DB is current


#### Queries Run For Stats 1 Listing 

In [7]:
file_query = "select count(*) from droid_ids"
size_query = 'select sum(size) from droid_ids'
extension_query = "select count(distinct file_extension) from droid_ids"
format_query = 'select count() from droid_formats'

total_files = conn.execute(text(file_query)).fetchone()[0]
total_size = conn.execute(text(size_query)).fetchone()[0]
total_extensions = conn.execute(text(extension_query)).fetchone()[0]
total_formats = conn.execute(text(format_query)).fetchone()[0]

print(f'1. Total Files: {format_number(total_files)}')
print(f'2. Total Size: {format_number(total_size)} bytes')
print(f'3. Total File Extensions: {format_number(total_extensions)}')
print(f'4. Total File Formats: {format_number(total_formats)}')

del file_query
del size_query
del extension_query
del format_query
del total_files
del total_size
del total_extensions
del total_formats

1. Total Files: 10,493,160
2. Total Size: 16,082,869,474,251 bytes
3. Total File Extensions: 4,982
4. Total File Formats: 715


### Define your queries below

In [2]:
# Find the number of unhashed PNG files
query = 'select count(*) from droid_ids where file_extension = "png" and hash = ""'

In [3]:
# Find the number of distinct file formats
query = 'select count(distinct file_format_name) from droid_formats'

In [41]:
# Select the distinct file format names
query = 'select distinct file_format_name from droid_formats'

In [10]:
# Find the 20 most common file formats and their occurences
query = 'select file_format_name, count() from droid_formats group by file_format_name order by count() desc limit 20'

In [87]:
# Find the files that have more than one format
query = 'select droid_ids.id, filename, file_extension, size from droid_ids join droid_formats on droid_ids.id = droid_formats.file_id group by file_id having count(file_id) > 1 order by droid_ids.size desc'

In [15]:
# Find the count of unhashed files
query = 'select * from droid_ids where hash = "" and type != "Folder"'

In [17]:
# Find the count of unhashed files and folders
query = 'select count() from droid_ids where hash = ""'

In [19]:
# Find the count of folders
query = 'select count() from droid_ids where type = "Folder"'

In [20]:
# Count the number of total records
query = 'select count() from droid_ids'

In [38]:
# Find all AVI files and order them by size
query = 'select min(droid_ids.size), (sum(droid_ids.size) / count()) / (1024 * 1024), max(droid_ids.size) / (1024 * 1024 * 1024) from droid_ids join droid_formats on droid_ids.id = droid_formats.file_id where (file_extension="avi" or file_extension="mp4") order by size desc'

In [145]:
# List the size of all the projects in descending total size
query = 'select filename, file_path, sum(size) from droid_ids group by project_name order by size desc limit 100'

In [32]:
# Find the Most occuring file extensions and their average sizes
query = 'select file_extension, sum(size), count(), sum(size) / count() as "average" from droid_ids join droid_formats on droid_ids.id = droid_formats.file_id where type != "Folder" group by file_extension order by count() desc'

In [26]:
# Look at the most occuring files's hashes and their size
query = 'select avg(size), hash, count() from droid_formats join droid_ids on droid_ids.id = droid_formats.file_id where type = "File" group by hash order by count() desc limit 20'

In [24]:
# Grab any projects that have the default team document, using size. TODO: Make this filter by hash
query = 'select count() from (select project_name from droid_ids where (filename="Team.doc" or filename="Team.docx") and (size=599 or size=37407) group by project_name)'

In [5]:
# Find all of the projects that do not match the past projects listing
query = 'select project_name, file_path from droid_ids where project_name like "Unverified Name%" group by project_name'

In [66]:
# File Format Percentage Throughout All Projects for the matplotlib code
s = text('select file_format_name, count() from droid_ids join droid_formats on droid_ids.id = droid_formats.file_id where type = "File" group by file_format_name order by count() desc')
file_format_name_population = conn.execute(s).fetchall()

In [3]:
# Recreation of table 1 from original analysis. Top 20 extensions by count
query = 'select file_extension, file_format_name, file_format_version, count() from droid_ids join droid_formats on droid_ids.id=droid_formats.file_id where type="File" and file_format_name != "" group by file_extension, file_format_name, file_format_version order by count() desc limit 20'
table_headers = ['Extension', 'File Format', 'Format Version', 'Count']

In [1]:
# Recreation of table 2 from original analysis. Top 20 known file formats by aggregrate size
query = 'select file_format_name, file_format_version, round(sum(size) / 1073741824.0, 2) from droid_ids join droid_formats on droid_ids.id=droid_formats.file_id where type="File" and file_format_name != "" group by file_format_name, file_format_version order by sum(size) desc limit 20'
table_headers = ['File Format Name', 'Version', 'Size (GB)']

In [14]:
query = 'select sum(size), count() from (select size, hash from droid_ids where (type="File" or type="Container") group by hash)'

In [4]:
query = 'select * from (select count() as count, sum(size) as size, filename, file_path from droid_ids group by file_path, type, hash order by count desc) where count > 1'

In [12]:
query = 'select count() from droid_ids where file_path="/"'

In [12]:
query = 'select type from droid_ids group by type'

In [8]:
query = 'select count() from droid_ids where (type="File" or type="Container")'

In [3]:
query = 'select project_name, project_year, project_semester from droid_ids group by project_name, project_year order by project_year desc, project_semester desc'


In [8]:
query = 'select count() from (select file_extension from droid_ids where file_format_count=0 group by file_extension)'

In [10]:
query = 'select count() from (select file_extension from droid_ids group by file_extension)'


In [12]:
query = 'select count(distinct file_format_name) from droid_formats'

In [3]:
query = 'select file_extension, count() from droid_ids where type != "Folder" and file_format_count=0 group by file_extension order by count() desc limit 100'
table_headers = ['File Extension', 'Count']

In [10]:
query = 'select file_path from droid_ids where type!="Folder" and file_format_count=0'

In [6]:
query = 'query = "select count() from droid_formats where file_format_name != """'

In [30]:
query = 'select * from droid_ids join droid_formats on droid_ids.id=droid_foramts.file_id where file_path="/2007_semester_3/Art Studio/RenderFarm/LBE/ErieShotFive_1_depth.png" '

In [32]:
query = 'select * from droid_ids where file_extension="lwf"'

In [ ]:
query = 'select file_extension, file_path from droid_ids where file_format_count=0 and project_name="Interact"'

In [19]:
query = 'select file_extension, file_path from droid_ids where file_format_count = 0 and file_extension="aspx" limit 50'

In [37]:
query = 'select sum(total_size), sum(total_count), sum(count) from (select count() as total_count, (count() - 1) as count, sum(size) as total_size from droid_ids where type != "Folder" group by hash, type) where total_count > 1'

In [41]:
query = 'select file_path, size from droid_ids where file_extension="avi" order by size desc limit 10'

In [8]:
query = 'select "total", sum(count), sum(total_size) from (select file_extension, count() as count, sum(size) as total_size from droid_ids where type != "Folder" and file_extension in ("exe", "apk", "ahk", "run", "xbe", "cmd", "jar", "ipa", "0xe", "bin", "elf", "xap", "air", "bat", "gadget", "ac", "app", "scr", "e_e", "u3p", "com", "out", "xlm", "pif", "widget", "script", "gpe", "73k", "command", "cgi", "x86_64", "rxe", "scb", "scar", "paf.exe", "sk", "ex_", "xex", "tcp", "epk", "acc", "ebs2", "nexe", "celx", "vxp", "coffee", "rfu", "isu", "fxp", "appimage", "ecf", "89k", "action", "xbap", "rbf", "x86", "plsc") group by file_extension order by count() desc)'
table_headers = ['File Extension', 'Count', 'Size']

In [7]:
query = 'select file_extension, count() as count, sum(size) as total_size from droid_ids where type != "Folder" and file_extension in ("swf", "swd", "fla") group by file_extension order by count() desc'
table_headers = ['File Extension', 'Count', 'Size']

In [3]:
query = 'select count() from droid_ids where type="Folder" and status="Empty"'

## The following cells are for executing a query defined above.

## You can either execute to the console or a file


## Execute a query to stdout

In [4]:
print('Trying to execute query:', query, flush=True)
s = text(query)
result = conn.execute(s).fetchall()
for line in result:
    print(line)
print('End')

Trying to execute query: select count() from droid_ids where type="Folder" and status="Empty"
(141550,)
End


## Execute a query into a file

In [6]:

print('Trying to execute query:', query, flush=True)
s = text(query)
stmt = f'Query:{s}'
result = conn.execute(s).fetchall()

Trying to execute query: select count() from droid_ids where type="Folder" and status="Empty"


In [21]:
with open('output.txt', 'w', encoding='utf-8') as file:
    file.write(stmt + '\n\n')
    for line in result:
        insert = ' '.join([str(item) for item in line])
        file.write(  + '\n')

# Execute a query into a table

In [8]:
result = conn.execute(text(query)).fetchall()
table_values = []
for line in result:
    table_values.append(list(line))
output_as_markdown_table(table_headers, table_values)

| File Extension | Count |     Size    |
| -------------- | ----- | ----------- |
|      swf       | 14672 |  8430577282 |
|      fla       |  6009 | 54359623148 |
|      swd       |  180  |   33059454  |



# Turn query results into a Graph with matplot


In [21]:
%matplotlib inline

In [21]:
#init_datatables_mode()

file_format_names = []
file_format_count = []
file_format_names_1 = []

for elem in file_format_name_population:
    if elem[1] > 1000:
        file_format_names.append(elem[0])
        file_format_count.append(elem[1])
    else:
        file_format_names_1.append(elem[0])
        
minority_count = len(file_format_names_1)

if minority_count > 1:
    file_format_names.append('Other')
    file_format_count.append(minority_count)
    
def func(pct, allvalues):
    absolute = int(pct / 100.*np.sum(allvalues))
    return "{:.1f}%\n({:d})".format(pct, absolute)

def queryPieGraph(values,headers,title_graph,title_legend):
    fig, ax = plt.subplots(figsize =(10, 7))
    wedges, texts, autotexts = ax.pie(file_format_count,
                                      autopct = lambda pct: func(pct, values),
                                      startangle = 90)

    # Adding legend
    ax.legend(wedges, headers,
              title = title_legend,
              loc ="center left",
              bbox_to_anchor =(1, 0, 0.5, 1))

    plt.setp(autotexts, size = 8, weight ="bold")
    ax.set_title(title_graph)

    plt.show()

#### Replication of Table 1, File Formats By Number from Kaltman 2019

| Extension |                 File Format                 | Format Version | Count  |
| --------- | ------------------------------------------- | -------------- | ------ |
|    bin    |                 Binary File                 |                | 618453 |
|    png    |          Portable Network Graphics          |      1.0       | 587617 |
|           |          Portable Network Graphics          |      1.0       | 281606 |
|    jpg    |         JPEG File Interchange Format        |      1.01      | 233531 |
|    xml    |          Extensible Markup Language         |      1.0       | 228147 |
|    jpg    |         JPEG File Interchange Format        |      1.02      | 185052 |
|    tif    |           Tagged Image File Format          |                | 182736 |
|    php    |     Extensible Hypertext Markup Language    |      1.0       | 180855 |
|    png    |          Portable Network Graphics          |      1.1       | 176427 |
|    html   |     Extensible Hypertext Markup Language    |      1.0       | 146563 |
|   class   |          Java Compiled Object Code          |                | 129911 |
|     js    |               JavaScript file               |                | 114332 |
|    exr    |                   OpenEXR                   |       2        | 106891 |
|    wav    |        Waveform Audio (PCMWAVEFORMAT)       |                | 105623 |
|    dll    |         Windows Portable Executable         |     32 bit     | 92153  |
|    png    |          Portable Network Graphics          |      1.2       | 87221  |
|    info   |        Guymager Acquisition Info File       |                | 76318  |
|    iff    |             Maya IFF Image File             |                | 74963  |
|     py    |              Python Script File             |                | 63176  |
|    jpg    | Exchangeable Image File Format (Compressed) |     2.2.1      | 60505  |

#### Replication of Table 2, File Formats By Size from Kaltman 2019

|               File Format Name              |   Version    | Size (GB) |
| ------------------------------------------- | ------------ | --------- |
|                     M2TS                    |              |  2428.36  |
|                  Quicktime                  |              |  2401.49  |
|        Audio/Video Interleaved Format       |              |  1903.86  |
|              MPEG-4 Media File              |              |   872.33  |
|               Adobe Photoshop               |              |   502.17  |
|           Tagged Image File Format          |              |   418.25  |
|            Maya ASCII File Format           |              |   397.34  |
|          Portable Network Graphics          |     1.0      |   340.53  |
|                   OpenEXR                   |      2       |   322.61  |
|                Macromedia FLV               |      1       |   187.17  |
| Exchangeable Image File Format (Compressed) |    2.2.1     |   154.24  |
|             Maya IFF Image File             |              |   133.54  |
|       Microsoft Powerpoint for Windows      | 2007 onwards |   121.23  |
|         JPEG File Interchange Format        |     1.02     |   114.42  |
|        Waveform Audio (PCMWAVEFORMAT)       |              |   104.1   |
|               Raw JPEG Stream               |              |   103.19  |
|          Java Compiled Object Code          |              |    98.9   |
|         Windows Portable Executable         |    32 bit    |    96.4   |
|            MPEG-2 Program Stream            |              |   95.43   |
|          Apple Lossless Audio Codec         |              |   92.36   |
